In [262]:
#use Pandas to import data
import pandas as pd
import zipfile
import requests
import io
import time
import numpy as np
from datetime import datetime
import re
from dateutil.parser import parse

In [2]:
## information on filename
citidir = "https://s3.amazonaws.com/tripdata/"
citi_extension = "-citibike-tripdata"
year = 2015
months = ['01','02','03','04','05','06','07','08','09','10','11','12']

In [3]:
## store all data first in dictionary format using months as keys
d = {}

## download monthly data from the web for each month
for mth in months:
    #create filenames for given month
    filename = str(year) + mth + citi_extension
    citibike_zip_filename = citidir + filename + ".zip"
    citibike_csv_filename = filename + ".csv"
    print(citibike_zip_filename)
    
    #find file and use read_csv to save in dictionary of dataframes
    requested_file = requests.get(citibike_zip_filename)
    z = zipfile.ZipFile(io.BytesIO(requested_file.content))
    d[mth] = pd.read_csv(z.open(citibike_csv_filename), header=0, sep=',', quotechar='"')

https://s3.amazonaws.com/tripdata/201501-citibike-tripdata.zip
https://s3.amazonaws.com/tripdata/201502-citibike-tripdata.zip
https://s3.amazonaws.com/tripdata/201503-citibike-tripdata.zip
https://s3.amazonaws.com/tripdata/201504-citibike-tripdata.zip
https://s3.amazonaws.com/tripdata/201505-citibike-tripdata.zip
https://s3.amazonaws.com/tripdata/201506-citibike-tripdata.zip
https://s3.amazonaws.com/tripdata/201507-citibike-tripdata.zip
https://s3.amazonaws.com/tripdata/201508-citibike-tripdata.zip
https://s3.amazonaws.com/tripdata/201509-citibike-tripdata.zip
https://s3.amazonaws.com/tripdata/201510-citibike-tripdata.zip
https://s3.amazonaws.com/tripdata/201511-citibike-tripdata.zip
https://s3.amazonaws.com/tripdata/201512-citibike-tripdata.zip


In [4]:
## Concatenate into single yearly dataframe
df = pd.DataFrame()

## append results of each month
for mth in months:
    df = df.append(d[mth], ignore_index=True)

In [6]:
#check out headers
print(df.tail())

         tripduration            starttime             stoptime  \
9937964           509  12/31/2015 16:50:17  12/31/2015 16:58:46   
9937965           591  12/31/2015 17:17:52  12/31/2015 17:27:43   
9937966           197  12/31/2015 17:56:22  12/31/2015 17:59:39   
9937967           372  12/31/2015 21:20:38  12/31/2015 21:26:50   
9937968           395  12/31/2015 21:49:19  12/31/2015 21:55:55   

         start station id          start station name  start station latitude  \
9937964              3242  Schermerhorn St & Court St               40.691029   
9937965              3242  Schermerhorn St & Court St               40.691029   
9937966              3242  Schermerhorn St & Court St               40.691029   
9937967              3242  Schermerhorn St & Court St               40.691029   
9937968              3242  Schermerhorn St & Court St               40.691029   

         start station longitude  end station id             end station name  \
9937964               -73.991

In [7]:
print("Median trip duration:", df["tripduration"].median())

Median trip duration: 629.0


In [8]:
#add new column to dataframe indicating whether start and end station IDs are the same or not
df['same_start_end'] = pd.Series(df["start station id"] == df["end station id"], index=df.index)
same_frac = df['same_start_end'].value_counts(normalize = True)
print("Fraction of rides that start and end at same station:", same_frac[True])

Fraction of rides that start and end at same station: 0.0223583913373


In [27]:
### count number of unique biks
unique_bikes = df['bikeid'].unique()
print(len(unique_bikes))

8477


In [61]:
bikes = df.groupby('bikeid')

In [78]:
visited_stations = {}

for k,v in bikes:
    start_stations = bikes.get_group(k)['start station id']
    end_stations = bikes.get_group(k)['end station id']
    stations = pd.concat([start_stations, end_stations], axis=0, ignore_index = True)
    unique_stations = stations.unique()
    visited_stations[k] = len(unique_stations)

In [81]:
np.std(list(visited_stations.values()))

54.541896535851848

In [93]:
len(vis)

8477

In [104]:
vis = list(visited_stations.values())
print(np.mean(vis))
print(np.median(vis))
print(np.std(vis))
np.sort(vis)[-1000:]

320.790020054
333.0
54.5418965359


array([355, 355, 355, 355, 355, 355, 355, 355, 355, 355, 355, 355, 355,
       355, 355, 355, 355, 355, 355, 355, 355, 355, 355, 355, 355, 355,
       355, 355, 355, 355, 355, 355, 355, 355, 355, 355, 355, 355, 355,
       355, 355, 355, 355, 355, 355, 355, 355, 355, 355, 355, 355, 355,
       355, 355, 355, 355, 355, 355, 355, 355, 355, 355, 355, 355, 356,
       356, 356, 356, 356, 356, 356, 356, 356, 356, 356, 356, 356, 356,
       356, 356, 356, 356, 356, 356, 356, 356, 356, 356, 356, 356, 356,
       356, 356, 356, 356, 356, 356, 356, 356, 356, 356, 356, 356, 356,
       356, 356, 356, 356, 356, 356, 356, 356, 356, 356, 356, 356, 356,
       356, 356, 356, 356, 356, 356, 356, 356, 356, 356, 356, 356, 356,
       356, 356, 356, 356, 356, 356, 356, 356, 356, 356, 356, 356, 356,
       356, 357, 357, 357, 357, 357, 357, 357, 357, 357, 357, 357, 357,
       357, 357, 357, 357, 357, 357, 357, 357, 357, 357, 357, 357, 357,
       357, 357, 357, 357, 357, 357, 357, 357, 357, 357, 357, 35

In [295]:
df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,same_start_end
0,1346,1/1/2015 0:01,1/1/2015 0:24,455,1 Ave & E 44 St,40.750020,-73.969053,265,Stanton St & Chrystie St,40.722293,-73.991475,18660,Subscriber,1960.0,2,False
1,363,1/1/2015 0:02,1/1/2015 0:08,434,9 Ave & W 18 St,40.743174,-74.003664,482,W 15 St & 7 Ave,40.739355,-73.999318,16085,Subscriber,1963.0,1,False
2,346,1/1/2015 0:04,1/1/2015 0:10,491,E 24 St & Park Ave S,40.740964,-73.986022,505,6 Ave & W 33 St,40.749013,-73.988484,20845,Subscriber,1974.0,1,False
3,182,1/1/2015 0:04,1/1/2015 0:07,384,Fulton St & Waverly Ave,40.683178,-73.965964,399,Lafayette Ave & St James Pl,40.688515,-73.964763,19610,Subscriber,1969.0,1,False
4,969,1/1/2015 0:05,1/1/2015 0:21,474,5 Ave & E 29 St,40.745168,-73.986831,432,E 7 St & Avenue A,40.726218,-73.983799,20197,Subscriber,1977.0,1,False


In [324]:
date_format_noseconds = '%m/%d/%Y %H:%M'
date_format_seconds = '%m/%d/%Y %H:%M:%S'
switch_indices = [824308, 2438684, 3379903]

In [330]:
#January-March and Jun don't include seconds on starttime - rely on switch_indices to change import formats
dates_1 = df['starttime'][:switch_indices[0]].apply(lambda x: datetime.strptime(x, date_format_noseconds))
dates_2 = df['starttime'][switch_indices[0]:switch_indices[1]].apply(lambda x: datetime.strptime(x, date_format_seconds))
dates_3 = df['starttime'][switch_indices[1]:switch_indices[2]].apply(lambda x: datetime.strptime(x, date_format_noseconds))
dates_4 = df['starttime'][switch_indices[2]:].apply(lambda x: datetime.strptime(x, date_format_seconds))

In [331]:
df['datetime'] = pd.concat([dates_1,dates_2,dates_3,dates_4], axis=0)

In [332]:
print(df.head())

   tripduration      starttime       stoptime  start station id  \
0          1346  1/1/2015 0:01  1/1/2015 0:24               455   
1           363  1/1/2015 0:02  1/1/2015 0:08               434   
2           346  1/1/2015 0:04  1/1/2015 0:10               491   
3           182  1/1/2015 0:04  1/1/2015 0:07               384   
4           969  1/1/2015 0:05  1/1/2015 0:21               474   

        start station name  start station latitude  start station longitude  \
0          1 Ave & E 44 St               40.750020               -73.969053   
1          9 Ave & W 18 St               40.743174               -74.003664   
2     E 24 St & Park Ave S               40.740964               -73.986022   
3  Fulton St & Waverly Ave               40.683178               -73.965964   
4          5 Ave & E 29 St               40.745168               -73.986831   

   end station id             end station name  end station latitude  \
0             265     Stanton St & Chrystie St    

In [341]:
df['startmonth'] = df['datetime'].apply(lambda x: x.month)

In [342]:
df_by_month = df.groupby('startmonth')

In [346]:
mean_duration_by_month = list(df_by_month['tripduration'].mean())

In [350]:
print(np.max(mean_duration_by_month)-np.min(mean_duration_by_month))

430.57029597


In [351]:
tst = df_by_month['tripduration'].mean()

In [353]:
np.min(tst)

649.38320723099582

In [334]:
def group_by_month(df, ind, col):
    return df[col].loc[ind].month

In [335]:
df_by_month = df.groupby(lambda x: group_by_month(df, x, 'datetime'))

KeyError: 'the label [430] is not in the [index]'

In [315]:
startindex = 2438684

for i, date in enumerate(df['starttime'][startindex:]):
    try:
        datetime.strptime(date, date_format_noseconds)
    except:
        print(i+startindex)
        time.sleep(1)

3379903
3379904
3379905
3379906


KeyboardInterrupt: 

In [314]:
tst = df['starttime'][2438684:].apply(lambda x: datetime.strptime(x, date_format_noseconds))

ValueError: unconverted data remains: :03

In [322]:
tst = df['starttime'][3379903:].apply(lambda x: datetime.strptime(x, date_format_seconds))

In [323]:
print(tst[-10:])

9937959   2015-12-31 12:00:05
9937960   2015-12-31 13:44:49
9937961   2015-12-31 15:05:11
9937962   2015-12-31 15:50:37
9937963   2015-12-31 15:50:52
9937964   2015-12-31 16:50:17
9937965   2015-12-31 17:17:52
9937966   2015-12-31 17:56:22
9937967   2015-12-31 21:20:38
9937968   2015-12-31 21:49:19
Name: starttime, dtype: datetime64[ns]


In [145]:
df_baby['datetime'] =  datetime.strptime(df['starttime'],'%m/%d/%Y %H:%M')

TypeError: strptime() argument 1 must be str, not Series

In [285]:
df.loc[825000]

tripduration                                          249
starttime                               4/1/2015 06:49:09
stoptime                                4/1/2015 06:53:19
start station id                                      481
start station name                   S 3 St & Bedford Ave
start station latitude                            40.7126
start station longitude                          -73.9626
end station id                                        539
end station name           Metropolitan Ave & Bedford Ave
end station latitude                              40.7153
end station longitude                            -73.9602
bikeid                                              20654
usertype                                       Subscriber
birth year                                           1957
gender                                                  1
same_start_end                                      False
Name: 825000, dtype: object

In [277]:
for i,date in enumerate(df['starttime']):
    try:
        datetime.strptime(date,'%m/%d/%Y %H:%M') # returns DataFrame
    except:
        print(i)
        time.sleep(.2)

824308
824309
824310
824311
824312
824313
824314
824315
824316
824317
824318
824319
824320
824321
824322
824323
824324
824325
824326
824327
824328
824329
824330
824331
824332
824333
824334
824335
824336
824337
824338
824339
824340
824341
824342
824343
824344
824345
824346
824347
824348
824349
824350
824351
824352
824353
824354
824355
824356
824357
824358
824359
824360
824361
824362


KeyboardInterrupt: 

In [206]:
df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,same_start_end
0,1346,1/1/2015 0:01,1/1/2015 0:24,455,1 Ave & E 44 St,40.750020,-73.969053,265,Stanton St & Chrystie St,40.722293,-73.991475,18660,Subscriber,1960.0,2,False
1,363,1/1/2015 0:02,1/1/2015 0:08,434,9 Ave & W 18 St,40.743174,-74.003664,482,W 15 St & 7 Ave,40.739355,-73.999318,16085,Subscriber,1963.0,1,False
2,346,1/1/2015 0:04,1/1/2015 0:10,491,E 24 St & Park Ave S,40.740964,-73.986022,505,6 Ave & W 33 St,40.749013,-73.988484,20845,Subscriber,1974.0,1,False
3,182,1/1/2015 0:04,1/1/2015 0:07,384,Fulton St & Waverly Ave,40.683178,-73.965964,399,Lafayette Ave & St James Pl,40.688515,-73.964763,19610,Subscriber,1969.0,1,False
4,969,1/1/2015 0:05,1/1/2015 0:21,474,5 Ave & E 29 St,40.745168,-73.986831,432,E 7 St & Avenue A,40.726218,-73.983799,20197,Subscriber,1977.0,1,False


In [204]:
dates = df['starttime']
datelist = list(datelen)

In [215]:
type(datelist[9])

str

In [237]:
splitdates = []

for date in datelist:
    
    splitdates.append(re.split('\s|/',date))
    


In [238]:
splitdates[0]

['1', '1', '2015', '0:01']

In [241]:
dateyear = [ x[2] for x in splitdates]

In [258]:
len(dateyear)
checkyear = [ x == '2015' for x in dateyear ]

In [261]:
print(sum(checkyear))
print(len(dateyear))

9937969
9937969


In [207]:
dates[0]
secondhalf = lambda x: x[1]


In [188]:
tst4 = [ secondhalf(date) for date in dates ]
print(tst4[:100])

['0:01', '0:02', '0:04', '0:04', '0:05', '0:07', '0:07', '0:08', '0:09', '0:10', '0:10', '0:11', '0:13', '0:13', '0:14', '0:14', '0:14', '0:16', '0:17', '0:18', '0:18', '0:18', '0:19', '0:20', '0:21', '0:21', '0:21', '0:22', '0:22', '0:22', '0:23', '0:23', '0:23', '0:24', '0:25', '0:25', '0:26', '0:27', '0:27', '0:27', '0:27', '0:27', '0:27', '0:28', '0:28', '0:28', '0:28', '0:29', '0:29', '0:31', '0:32', '0:32', '0:32', '0:32', '0:32', '0:33', '0:34', '0:35', '0:35', '0:35', '0:35', '0:35', '0:35', '0:35', '0:36', '0:37', '0:37', '0:38', '0:38', '0:39', '0:39', '0:40', '0:41', '0:42', '0:42', '0:42', '0:43', '0:43', '0:43', '0:43', '0:44', '0:44', '0:44', '0:44', '0:46', '0:46', '0:46', '0:46', '0:46', '0:46', '0:47', '0:47', '0:47', '0:48', '0:48', '0:49', '0:49', '0:49', '0:49', '0:51']


In [192]:
tst5 = [x.split(":") for x in tst4]

In [194]:
tst6 = [ secondhalf(date) for date in tst5 ]


In [198]:
tst7 = [len(x) for x in tst6]

In [203]:
print(len(tst7))
print(tst7.count(2))

9937969
9937969


In [201]:
tst2 = [ lambda x: x[1][:] for x in dates ]

In [180]:
tst3 = list(tst2)
print(tst3[:100])

[<function <listcomp>.<lambda> at 0x16361a840>, <function <listcomp>.<lambda> at 0x16361af28>, <function <listcomp>.<lambda> at 0x16361ae18>, <function <listcomp>.<lambda> at 0x11f4ca0d0>, <function <listcomp>.<lambda> at 0x11f4cabf8>, <function <listcomp>.<lambda> at 0x11f4ca840>, <function <listcomp>.<lambda> at 0x11f4ca8c8>, <function <listcomp>.<lambda> at 0x11f4ca7b8>, <function <listcomp>.<lambda> at 0x11f4caa60>, <function <listcomp>.<lambda> at 0x11f4ca9d8>, <function <listcomp>.<lambda> at 0x11f4caf28>, <function <listcomp>.<lambda> at 0x11f4cad08>, <function <listcomp>.<lambda> at 0x11f4caae8>, <function <listcomp>.<lambda> at 0x11f4cae18>, <function <listcomp>.<lambda> at 0x11f4ca950>, <function <listcomp>.<lambda> at 0x1614fe048>, <function <listcomp>.<lambda> at 0x1614fe0d0>, <function <listcomp>.<lambda> at 0x1614fe158>, <function <listcomp>.<lambda> at 0x1614fe1e0>, <function <listcomp>.<lambda> at 0x1614fe268>, <function <listcomp>.<lambda> at 0x1614fe2f0>, <function <l

In [162]:
datelen.value_counts()

18    4715467
19    2039306
17    1417669
15     981406
14     678261
13     105860
Name: starttime, dtype: int64

In [70]:
type(end_stations)

pandas.core.series.Series

In [60]:
bikes['test_column'] = bikes['start station id']*2

TypeError: unsupported operand type(s) for *: 'SeriesGroupBy' and 'int'

In [13]:
start_stations = bikes['start station id']
end_stations = bikes['end station id']

In [57]:
for k,v in start_stations:
    print(type(k))
    print(type(v))
    time.sleep(3)

<class 'numpy.int64'>
<class 'pandas.core.series.Series'>
<class 'numpy.int64'>
<class 'pandas.core.series.Series'>
<class 'numpy.int64'>
<class 'pandas.core.series.Series'>


KeyboardInterrupt: 

In [22]:
for k,v in start_stations:
    print(k)
    print(len(v))
    time.sleep(3)

14529
1233
14530
588
14531
1081
14532
1088
14533
1326
14534
1308


KeyboardInterrupt: 

In [43]:
list(start_stations)[0][1]

1524        326
1793        497
5715        496
8123        195
8360        250
8642        358
8812        435
9156        466
29006       362
40268       456
41290       498
42450       474
46917       478
49233       520
52693       379
64762       468
66571       465
69884       348
70780       128
71709       508
74789       362
74959       167
92747       465
93800       359
94762       479
95286       512
95408       489
95589       515
95984       434
97689       357
           ... 
9673414     499
9673468     499
9683359     503
9691099     505
9702844     510
9704054     511
9704064     511
9710968     513
9718734     517
9721447     517
9727077     519
9732240     519
9738998     521
9740958     521
9742686     522
9748180     523
9767093     531
9771969     533
9775500     536
9778113     537
9785418     540
9786726     545
9856904    3134
9860888    3137
9860914    3137
9862644    3139
9891913    3160
9904603    3168
9912224    3175
9926818    3230
Name: start station id, 

In [47]:
bikes.get_group(14529)['start station id'].unique()

array([ 326,  497,  496,  195,  250,  358,  435,  466,  362,  456,  498,
        474,  478,  520,  379,  468,  465,  348,  128,  508,  167,  359,
        479,  512,  489,  515,  434,  357,  469,  490,  352, 2022,  447,
        495,  237,  402,  280,  342,  531,  296,  249,  540,  476,  393,
        300,  336, 2010,  368,  463,  127,  382,  223,  293,  403,  317,
        151,  238,  477,  526,  488,  380,  455,  267,  513,  404,  327,
        521,  533,  507,  236,  325,  459,  426,  347,  294,  523,  546,
        445,  517,  537,  464,  433,  312,  518,  164,  538,  458,  530,
        328,  318, 2006,  499,  461,  334,  264,  504,  411,  301,  491,
        511, 2012,  501,  522,  510,  441,  485,  305,  480,  484,  446,
       2021,  310,  406,  376,  146,  408,  505,  375,  536,  525,  448,
        259,  400,  303,  152,  306,  276,  316,  415,  279, 2004,  252,
        503,  516,  308,  363,  346,  383,  492,  487,  168,  527,  509,
        483,  297,  438,  401,  356,  229,   79,  2

In [39]:
start_groups = start_stations[:][1].groups

Exception: Column(s) start station id already selected

In [28]:
for k,v in start_groups.items():
    print(k)
    print(v)
    time.sleep(3)

16384
Int64Index([ 310666,  310792,  310827,  311876,  311957,  312174,  312284,
             317817,  327540,  329184,
            ...
            9871878, 9876316, 9887862, 9891524, 9894988, 9900518, 9901120,
            9918194, 9926187, 9929251],
           dtype='int64', length=1261)
16385
Int64Index([  57492,   57562,   59694,   60688,   63911,   64175,   67342,
              70718,  150859,  154102,
            ...
            9864457, 9864462, 9867593, 9873595, 9878460, 9881575, 9907128,
            9909383, 9918169, 9933977],
           dtype='int64', length=980)


KeyboardInterrupt: 

In [25]:
len(start_groups.keys())

8477

In [355]:
print(df.head())

   tripduration      starttime       stoptime  start station id  \
0          1346  1/1/2015 0:01  1/1/2015 0:24               455   
1           363  1/1/2015 0:02  1/1/2015 0:08               434   
2           346  1/1/2015 0:04  1/1/2015 0:10               491   
3           182  1/1/2015 0:04  1/1/2015 0:07               384   
4           969  1/1/2015 0:05  1/1/2015 0:21               474   

        start station name  start station latitude  start station longitude  \
0          1 Ave & E 44 St               40.750020               -73.969053   
1          9 Ave & W 18 St               40.743174               -74.003664   
2     E 24 St & Park Ave S               40.740964               -73.986022   
3  Fulton St & Waverly Ave               40.683178               -73.965964   
4          5 Ave & E 29 St               40.745168               -73.986831   

   end station id             end station name  end station latitude  \
0             265     Stanton St & Chrystie St    

In [358]:
users = df.groupby('usertype')

In [375]:
max_ride_time = {'Customer': 1800, 'Subscriber': 2700}
ride_number = {}
rides_overtime = {}

for k,v in users:

    ride_times = users.get_group(k)['tripduration']
    ride_number[k] = len(ride_times)
    rides_overtime[k] = np.sum(ride_times > max_ride_time[k])
    
overtime_frac = np.sum(list(rides_overtime.values()))/np.sum(list(ride_number.values()))


In [376]:
print(overtime_frac)

0.0381067801681


In [390]:
for (k1,v1),(k2,v2) in zip(ride_number.items(),rides_overtime.items()):
    print(k1)
    print(v2/v1)

Customer
0.232040575568
Subscriber
0.00862699930147


In [377]:
rides_overtime.values()

dict_values([304282, 74422])

In [378]:
ride_number.values()

dict_values([1311331, 8626638])

In [380]:
np.sum(list(rides_overtime.values()))

378704

In [381]:
np.sum(list(ride_number.values()))

9937969